In [ ]:
#r "nuget:Microsoft.Spark"

In [ ]:
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

// run spark : spark-submit --class org.apache.spark.deploy.dotnet.DotnetRunner --master local D:\3bStudio\Sandbox\spark-program\FirstSparkProgram\bin\Debug\net6.0\microsoft-spark-3-0_2.12-2.1.0.jar debug
// see Spark portal at : http://localhost:4040
var spark = SparkSession
    .Builder()
    .AppName("spark-streaming")
    .GetOrCreate();

spark.SparkContext.SetLogLevel("WARN");

In [ ]:
// Before running this step, start NC using this : nc -vvv -l -p 9999
DataFrame lines = spark
    .ReadStream()
    .Format("socket")
    .Option("host", "localhost")
    .Option("port", 9999)
    .Option("includeTimestamp", true)
    .Load();

In [ ]:
DataFrame words = lines
            .Select(Explode(Split(lines["value"], " "))
            .Alias("word"));

DataFrame wordCounts = words.GroupBy("word").Count();

In [ ]:
var query = wordCounts
    .WriteStream()
    .OutputMode("complete")
    .Format("console")
    .Start();

In [ ]:
query.AwaitTermination();